In [18]:
from typing import Annotated, Sequence, TypedDict, List, Dict
from dotenv import load_dotenv  
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END, START
from langchain_groq import ChatGroq
from langgraph.prebuilt import ToolNode
import os
import json
import requests
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import MSO_ANCHOR
from io import BytesIO


In [19]:
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

llm = ChatGroq(model="llama-3.1-8b-instant")

In [20]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    subtopic: List[str]
    slide_segments: List[Dict[str, str]]
    ppt_output_path: str
    audio_output_path: List[str]

In [21]:
def load_json_to_agent_state(json_path: str) -> AgentState:
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    complete_slide_segments = []
    for slide in data.get('slide_segments', []):
        complete_slide = {
            "slide_no": slide.get("slide_no", 0),
            "subtopic": slide.get("subtopic", ""),
            "content_to_display": slide.get("content_to_display", ""),
            "narration_script": slide.get("narration_script", ""),
            "is_blank_slide": slide.get("is_blank_slide", False),
            "image_address": slide.get("image_address", ""),
            "video_address": slide.get("video_address", ""),
            "image_position": slide.get("image_position", ""),
            "content_position": slide.get("test_position", "")
        }
        complete_slide_segments.append(complete_slide)

    return AgentState(
        messages=[],
        subtopic=data.get('subtopics', []),
        slide_segments=complete_slide_segments,
        ppt_output_path=""
    )


In [ ]:
from groq import Groq

# Initialize the Groq client
groq_client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

def narration_to_audio(state: AgentState) -> AgentState:
    updated_audio_paths = []
    output_dir = "../assets/audio"
    os.makedirs(output_dir, exist_ok=True)

    for slide in state['slide_segments']:
        slide_no = slide["slide_no"]
        narration = slide["narration_script"]

        if not narration:
            updated_audio_paths.append("")
            continue

        speech_file_path = os.path.join(output_dir, f"slide{slide_no}.wav")
        
        try:
            response = groq_client.audio.speech.create(
                model="playai-tts",
                voice="Fritz-PlayAI", 
                input=narration,
                response_format="wav"
            )
            response.write_to_file(speech_file_path)
            updated_audio_paths.append(speech_file_path)
        except Exception as e:
            print(f"[ERROR] Failed to generate audio for slide {slide_no}: {e}")
            updated_audio_paths.append("")

    state["audio_output_path"] = updated_audio_paths
    return state


In [28]:
def load_data_node(state: AgentState) -> AgentState:
    return load_json_to_agent_state('../assets/scripts/slide_segments.json')

In [29]:
graph = StateGraph(AgentState)

graph.add_node("LoadData", load_data_node)
graph.add_node("NarrationToAudio", narration_to_audio)

graph.add_edge(START, "LoadData")
graph.add_edge("LoadData", "NarrationToAudio")
graph.add_edge("NarrationToAudio", END)


compiled_graph = graph.compile()


In [30]:
final_state = compiled_graph.invoke({})
print("Generated Audio Paths:", final_state['audio_output_path'])

[ERROR] Failed to generate audio for slide 1: Error code: 400 - {'error': {'message': 'The model `playai-tts` requires terms acceptance. Please have the org admin accept the terms at https://console.groq.com/playground?model=playai-tts', 'type': 'invalid_request_error', 'code': 'model_terms_required'}}
[ERROR] Failed to generate audio for slide 2: Error code: 400 - {'error': {'message': 'The model `playai-tts` requires terms acceptance. Please have the org admin accept the terms at https://console.groq.com/playground?model=playai-tts', 'type': 'invalid_request_error', 'code': 'model_terms_required'}}
[ERROR] Failed to generate audio for slide 3: Error code: 400 - {'error': {'message': 'The model `playai-tts` requires terms acceptance. Please have the org admin accept the terms at https://console.groq.com/playground?model=playai-tts', 'type': 'invalid_request_error', 'code': 'model_terms_required'}}
[ERROR] Failed to generate audio for slide 4: Error code: 400 - {'error': {'message': 'T